# Una introducción al procesamiento del lenguaje natural (NLP) en Python

**"A veces pienso que mi vida no es más que un conjunto de cursos de introducción a..."**, así terminaba una clase sobre [docencia online](https://metadocencia.netlify.app/cursos/abc-online/intro-abc/), a la que asistí hace unas semanas. 
Después de varios días de seguir con esa frase en mente, abracé la idea y decidí animarme a escribir este breve texto de "Introducción al procesamiento del lenguaje natural en Python".

La idea es entonces dar una primera aproximación al tema en español, ya que la mayoría de los textos sobre el mismo se encuentran en inglés. Este es mi primer texto sobre programación, así que probablemente me avergüence de él dentro de unos meses, pero como dice el pensador contemporáneo ["Jake the dog"](https://www.youtube.com/watch?v=smgQiGABQMs):

![Jake the Dog](https://pbs.twimg.com/media/ChezoMMUkAAVy26?format=jpg&name=small)

A lo largo de este texto vamos a analizar los títulos de los posts de la página ["HackerNews"](https://news.ycombinator.com/) (una comunidad donde los usuarios suben sus artículos y otros pueden votarlos) e intentar predecir el número de votos que recibirán, dependiendo de las palabras utilizadas. Los pasos que vamos a seguir se podrían resumir de la siguiente manera:

1.   Importación e inspección inicial de la base de datos.
2.   Muestreo aleatorio de la base de datos.
3.   Implementación del modelo de "bolsa de palabras".
4.   Implementación del modelo de regresión para predecir el número de puntos del post.

Una primera duda que nos puede venir a la mente podría ser ¿qué es el procesamiento del lenguaje natural? Vamos a intentar esbozar una explicación fácil.

Cuando intentamos aprender un idioma nuevo tenemos que aprender nuevos significados de palabras, nuevas reglas gramaticales, nuevas pronunciaciones, etc. La primera clase de idiomas típicamente se basa en saludos y conversaciones simples con la idea de que, poco a poco, vayamos incorporando el significado de cada término y adoptemos lentamente la nueva gramática. Por esta razón empezamos tomando notas y traduciendo cada palabra a nuestra lengua materna. De manera análoga nosotros tenemos que "enseñarle" a la computadora cómo es nuestro idioma, qué reglas usamos y traducir nuestras palabras al "idioma" de las computadoras, que se basa en 0s y 1s (o más bien en existencia o ausencia de pulsos eléctricos). **En resumen podríamos decir que el procesamiento del lenguaje natural consiste en enseñarle a las computadoras a interpretar, entender y manipular el lenguaje humano.**

Empecemos con el primer paso del proyecto, la **importación e inspección inicial de la base de datos**. Los datos que vamos a utilizar son publicaciones realizadas por los usuarios de HackerNews entre 2006 y 2015. El mismo fue obtenido por Arnaud Drizard utilizando la API de la página y puede encontrarse en el siguiente [repositorio de Github](https://github.com/arnauddri/hn). De los archivos que se encuentran en el repositorio sólo trabajaremos con stories.csv (que pesa 171.MB aproximadamente) y que según la documentación tiene las siguientes columnas:

* id
* created_at
* created_at_i
* author
* points
* url_hostname
* num_comments
* title

Inicialmente importamos pandas, cargamos la base de datos e indicamos los nombres de las columnas (ya que el archivo no tiene títulos):

In [ ]:
import pandas as pd
submissions = pd.read_csv("stories.csv", header = None,
                          names = ["id", "created_at", "created_at_i", "author", "points",
                                   "url_hostname", "num_comments", "title"])

Ahora exploramos la estructura del Dataframe para ver cómo está compuesto en detalle:

In [ ]:
submissions.head()

,id,created_at,created_at_i,author,points,url_hostname,num_comments,title
0,9079978,2015-02-20T11:29:58.000Z,1424431798,Immortalin,2,NaN,0,Ask HN: Simple SaaS as first Golang web app?
1,9079983,2015-02-20T11:34:22.000Z,1424432062,Rutger24s,1,startupjuncture.com,0,24sessions: live business advice over video-chat
2,9079986,2015-02-20T11:35:32.000Z,1424432132,AndrewDucker,3,blog.erratasec.com,0,Some notes on SuperFish
3,9079988,2015-02-20T11:36:18.000Z,1424432178,davidiach,1,twitter.com,0,Apple Watch models could contain 29.16g of gold
4,9080000,2015-02-20T11:41:06.000Z,1424432466,CiaranR,1,phpconference.co.uk,0,PHP UK Conference Diversity Scholarship Programme


In [ ]:
submissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1553934 entries, 0 to 1553933
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   id            1553934 non-null  int64 
 1   created_at    1553934 non-null  object
 2   created_at_i  1553934 non-null  int64 
 3   author        1553934 non-null  object
 4   points        1553934 non-null  int64 
 5   url_hostname  1459195 non-null  object
 6   num_comments  1553934 non-null  int64 
 7   title         1550600 non-null  object
dtypes: int64(4), object(4)
memory usage: 94.8+ MB


Como podemos ver la base de datos está compuesta por más de un millón y medio de filas, como este texto sólo tiene un fin didáctico vamos proceder al segundo paso que es el **muestreo aleatorio de la base de datos**, que consistirá en tomar 6000 filas al azar para continuar con nuestro trabajo. Para el muestreo aleatorio vamos a usar un random_state de 1. La idea de esto es lograr que el proceso sea repetible y no especular repitiendo el muestreo hasta que obtengamos un resultado que favorezca a nuestro modelo.

In [ ]:
submissions = submissions.sample(6000, random_state = 1)

In [ ]:
submissions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6000 entries, 404493 to 826011
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            6000 non-null   int64 
 1   created_at    6000 non-null   object
 2   created_at_i  6000 non-null   int64 
 3   author        6000 non-null   object
 4   points        6000 non-null   int64 
 5   url_hostname  5631 non-null   object
 6   num_comments  6000 non-null   int64 
 7   title         5991 non-null   object
dtypes: int64(4), object(4)
memory usage: 421.9+ KB


Como podemos ver redujimos la base de datos a sólo 6000 entradas. Como próximo paso vamos a descartar algunas de las columnas de la base de datos, por ahora sólo nos quedaremos con:

* created_at
* points
* url_hostname
* title

In [ ]:
submissions = submissions.drop(["id", "created_at_i", "author", "num_comments"], axis = 1)

In [ ]:
submissions.head()

,created_at,points,url_hostname,title
404493,2013-09-26T12:11:48Z,4,youtube.com,Bill Gates interview at Harvard (2013)
1435421,2009-05-08T14:36:28Z,2,news.bbc.co.uk,Google boss won't quit Apple job
1207322,2011-02-15T18:44:58Z,1,n-rhman.com,
756837,2012-07-23T03:07:44Z,93,spectrum.ieee.org,Why Bad Jobs-or No Jobs-Happen to Good Workers
639885,2012-12-19T19:50:57Z,2,charlespetzold.com,First-Person Shooter


Para finalizar esta etapa de limpieza de datos vamos a descartar aquellas filas a las que les falta información, con el fin de que no afecten nuestro futuro modelo de predicción, incluyendo aquellos casos en los que no se incluye el título.

In [ ]:
submissions = submissions.dropna()

In [ ]:
submissions = submissions.drop(submissions[submissions["title"] == " "].index)

Finalmente reiniciamos el índice:

In [ ]:
submissions = submissions.reset_index()

In [ ]:
submissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5619 entries, 0 to 5618
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         5619 non-null   int64 
 1   created_at    5619 non-null   object
 2   points        5619 non-null   int64 
 3   url_hostname  5619 non-null   object
 4   title         5619 non-null   object
dtypes: int64(2), object(3)
memory usage: 219.6+ KB


Como dijimos anteriormente, las computadoras no "hablan" nuestro mismo idioma, por lo que vamos a necesitar "traducirlo" a su idioma. Para hacer esto vamos a convertir cada título en su representación numérica.

Como indicamos en el tercer paso del procedimiento a seguir, el modelo que vamos a utilizar es el de "bolsa de palabras" (bag of words en inglés), el cual representa cada pieza de texto como un vector numérico indicando el número de veces que una palabra se repite en una oración.

N° oración | hace | hoy | demasiado |mucho | frio | en | buenos | aires | para | andar | bicleta |
--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
1 | 1 | 0 | 0 | 1 | 1 | 1 | 1 | 1 | 0 | 0 | 0|
2 | 1 | 1 | 1 | 0 | 1 | 2 | 1 | 1 | 1 | 1 | 1|

El primer paso para crear una bolsa de palabras es tokenizar, lo que consiste en descomponer cada oración en palabras independientes.

Por ejemplo podemos tokenizar:
 "Hace mucho frío en Buenos Aires" y "Hoy hace demasiado frío en Buenos Aires para andar en bicicleta" como:
 
1. ["hace", "mucho", "frío", "en", "buenos", "aires"]
2. ["hoy", "hace", "demasiado", "frío", "en", "buenos", "aires", "para", "andar", "en", "bicicleta"]

Para lograr esto vamos a dividir cada oración en una lista individual de tokens, utilizando al espacio como separador. Sin embargo, antes de dividir los títulos es importante que le indiquemos a la computadora que "Aires", "aires" y "aires!" se refieren a la misma palabra, por lo que vamos a preprocesar los títulos eliminando las puntuaciones y pasando todo a minúscula, antes de "traducirlo". En resumen vamos a:

* Eliminar los signos de puntuación de los títulos (para esto vamos a hacer uso de expresiones regulares).
* Pasar todas las palabras a minúscula.
* Separar cada palabra y transformar los títulos en una lista.

In [ ]:
#Primero eliminamos la puntuacion
submissions["title"] = submissions["title"].str.replace("[^0-9a-zA-Z ]+"," ")

#Luego pasamos todo a minuscula
submissions["title"] = submissions["title"].str.lower()

#finalmente separamos cada palabra por los espacios y generamos una nueva columna
submissions["tokenized_headlines"] = submissions["title"].str.split()

*Nota sobre expresiones regulares*: Por si no están muy familiarizados con las expresiones regulares, lo que hicimos fue indicarle a la función que detecte todos los valores que no sean alfanuméricos ni espacios. El ^ niega todo lo que está entre corchetes, es decir que le estamos pidiendo a la computadora que busque y reemplace todos los valores que **NO** estén entre A-Z, ni entre a-z (para considerar mayúsculas y minúsculas), ni entre 0-9 (para considerar números). Luego agregamos a la lista de excluidos un espacio y fuera del corchete colocamos un + para indicar que repita esta condición una o más veces. En la segunda mitad de la función le indicamos que todos los caracteres que cumplan con esa condición (es decir que no estén incluidos en la lista de los **NO**) sean reemplazados por espacios “ “, ya que después lo usaremos como separador.

In [ ]:
submissions.head()

,index,created_at,points,url_hostname,title,tokenized_headlines
0,404493,2013-09-26T12:11:48Z,4,youtube.com,bill gates interview at harvard 2013,"[bill, gates, interview, at, harvard, 2013]"
1,1435421,2009-05-08T14:36:28Z,2,news.bbc.co.uk,google boss won t quit apple job,"[google, boss, won, t, quit, apple, job]"
2,756837,2012-07-23T03:07:44Z,93,spectrum.ieee.org,why bad jobs or no jobs happen to good workers,"[why, bad, jobs, or, no, jobs, happen, to, goo..."
3,639885,2012-12-19T19:50:57Z,2,charlespetzold.com,first person shooter,"[first, person, shooter]"
4,1164449,2011-04-15T20:26:31Z,5,whitehouse.gov,obama releases strategy for trusted identities...,"[obama, releases, strategy, for, trusted, iden..."


Ahora que ya tenemos nuestros tokens, podemos comenzar a convertir las oraciones en sus representaciones numéricas. Primero vamos a obtener todas las palabras únicas de los títulos. Luego vamos a crear una matriz y asignar esas palabras como nombres de columna, inicializando todos los valores en cero.
En este caso sólo utilizaremos palabras que se repitan más de una vez, ya que incluir palabras con una única aparición le brinda poca información al modelo y le introduce ruido innecesario.

In [ ]:
unique_tokens = set()
single_tokens = set()

def analyze_tokens(list):
    """Toma una lista y divide en tokens unicos (single_tokens) y tokens unicos con más de una repeticion (unique_tokens)
    
    Args:
        lista = toma una lista de tokens
        
    Returns:
        None = se incorpora en los sets externos
    """
    for token in list:
        if token in single_tokens:
            unique_tokens.add(token)
        else:
            single_tokens.add(token)
            

In [ ]:
tokens_analyzed = submissions["tokenized_headlines"].apply(analyze_tokens)

In [ ]:
print("Hay {} tokens unicos.".format(len(single_tokens)))
print("Hay {} tokens unicos que se repiten más de una vez.".format(len(unique_tokens)))
print("Solo un {:.2f}% de los tokens únicos se repite más de una vez.".format((len(unique_tokens)/len(single_tokens))*100))

Hay 10400 tokens unicos.
Hay 4114 tokens unicos que se repiten más de una vez.
Solo un 39.56% de los tokens únicos se repite más de una vez.


In [ ]:
import numpy as np

#Creamos una matriz con todos los tokens unicos como columnas
counts = pd.DataFrame(0, index = np.arange(len(submissions["tokenized_headlines"])),
                      columns=unique_tokens)

In [ ]:
counts.head()

,hijacking,original,adoption,interested,blocked,designs,won,ecommerce,principles,hairloss,english,demo,e3,paradox,off,adult,invented,cybersecurity,tevez,roles,ios,tv,ago,3,happiness,missing,billionaire,ipsum,99designs,feat,oem,thiel,50000,fixing,suggestions,leaves,kit,typography,ideas,investment,...,nike,scalability,em,720p,list,talk,lesson,successful,fitness,like,justice,authentication,halo,gaza,fights,ghost,comment,delete,joke,months,bans,canadian,client,yc,humanoid,action,protected,bet,space,exclusively,switch,children,downloads,managers,icrosofts,goliath,war,firefox,exist,coverage
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Ahora que tenemos una matriz donde todos los valores son cero, vamos a llenarla con los valores correctos en cada celda. Esto involucra ir a cada conjunto de tokens e incrementar los contadores en la columna apropiada.

In [ ]:
for i, tokens in enumerate(submissions["tokenized_headlines"]):
    for token in tokens:
        if token in unique_tokens:
            counts.loc[i,token] += 1

A esta altura tenemos más de 4000 columnas en nuestra matriz. El exceso de columnas también puede introducir ruido al modelo. Hay dos situaciones que pueden reducir la precisión de la predicción:
* Palabras que ocurren pocas de veces pueden generar un sobreajuste ([overfitting](https://en.wikipedia.org/wiki/Overfitting)) porque el modelo no tiene suficiente información para decidir si son importantes. A su vez estas palabras probablemente se correlacionen distinto en el set de entrenamiento y en el de prueba.
* Palabras que ocurren demasiadas veces también pueden generar problemas ya que ocurren prácticamente en todo título y no agregan información nueva que permita correlacionar con los votos positivos. En inglés estas palabras son llamadas ["stopwords"](https://en.wikipedia.org/wiki/Stop_words).

Para reducir el número de palabras a analizar y permitir que el modelo de regresión realice mejores predicciones, vamos a remover las palabras que ocurren menos de 5 veces o más de 100 veces.

In [ ]:
word_counts = counts.sum()

counts = counts[word_counts[(word_counts <= 100) & (word_counts >= 5)].index]

Ahora vamos a iniciar el 4to y último paso, la **implementación del modelo de regresión para predecir el número de puntos del post**. Para esto vamos a dividir la base de datos en dos sets y evaluar nuestro algoritmo, ajustándolo con los datos de entrenamiento y luego midiendo su eficiencia con los datos de prueba.

Para esto vamos a usar la función train_test_split() de scikit-learn. Utilizaremos como parámetros un 20% para el tamaño del set de prueba y un estado aleatorio = 1. Al igual que con el muestreo aleatorio el objetivo de definirlo en 1 es lograr que el modelo sea reproducible.

Los datos X_train y X_test van a contener los datos de entrada y y_train, y_test van a contener lo que queremos predecir (es decir, los votos positivos).

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, submissions["points"], test_size=0.2, random_state=1)

En este paso debemos seleccionar el modelo que utilizaremos para realizar las predicciones. Por una cuestión didáctica optaremos por un modelo simple como es la regresión lineal. Este modelo busca predecir el número de votos a través de una recta que minimice la distancia entre los votos reales y los predichos por el modelo como se puede ver a continuación:

![texto alternativo](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Linear_regression.svg/1200px-Linear_regression.svg.png)

Para esto haremos uso nuevamente de la librería scikit-learn y de la clase LinearRegression.

Primero vamos a inicializar el modelo usando dicha clase. Luego vamos a usar el método fit() para entrenar el modelo con X_train y y_train. Finalmente vamos a predecir los resultados utilizando X_test.

Cuando hacemos predicciones con un modelo de regresión lineal, el modelo asigna coeficientes a cada columna. Es decir que busca determinar cuáles palabras se correlacionan con más votos positivos y cuáles con menos. Encontrando estas correlaciones el modelo intentará predecir los votos positivos dependiendo de las palabras que se encuentren presente en el título.

In [ ]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression()

clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

Finalmente, utilizando nuestras predicciones vamos a calcular el error de nuestro modelo. Antes vamos a tener que elegir una métrica de error, en este caso vamos a trabajar con el error cuadratico medio (MSE) ya que penaliza los errores cuanto más lejos se encuentren del valor real, al elevarlos al cuadrado. Otra métrica de error posible a utilizar podría ser el MAE, la diferencia entre ambos excelede al presente texto, por lo que, para más información pueden ir al [siguiente artículo](https://medium.com/human-in-a-machine-world/mae-and-rmse-which-metric-is-better-e60ac3bde13d).

In [ ]:
mse = ((y_test - predictions)**2).sum()/len(predictions)

In [ ]:
print(mse)

2411.253069406075


Conociendo el valor de MSE podemos compararlo con la distribución de los votos positivos. Para eso obtendremos la [media](https://es.wikipedia.org/wiki/Mediana_(estad%C3%ADstica)) y la [desviación estandar](https://es.wikipedia.org/wiki/Desviaci%C3%B3n_t%C3%ADpica) de los votos positivos:

In [ ]:
mean_upvotes = submissions["points"].mean()
std_upvotes = submissions["points"].std()

In [ ]:
print("La media de votos es: {:.2f}".format(mean_upvotes))
print("La desviación estándar de votos es: {:.2f}".format(std_upvotes))
print("La raíz del error cuadrático medio es: {:.2f}".format(mse**0.5))

La media de votos es: 10.28
La desviación estándar de votos es: 39.82
La raíz del error cuadrático medio es: 49.10


Como podemos ver la media de los votos positivos es de 10,28 y la desviación estandar es de 39,82. Si tomamos la raiz cuadrada del MSE vemos que es 49,10, lo cual puede interpretarse como que nuestro error promedio está a 49,10 votos positivos de distancia del valor real. Se trata de un valor bastante elevado pero a su vez esperable, ya que era poco probable que el número de votos siga una distribución lineal, como dijimos anteriormente la elección de este modelo perseguía un objetivo didáctico y no de precisión.

Si bien vamos a finalizar el texto en este punto para evitar que se haga demasiado largo, podríamos tomar diferentes aproximaciones con el fin de mejorar la precisión del modelo, como por ejemplo:
* Usar otros algoritmos de predicción más potentes como por ejemplo "Random Forest", que permitan captar la no linealidad entre las palabras utilizadas y los votos obtenidos.
* Ampliar el número de muestras: esto debería reducir el error drásticamente ya que utilizar más información nos ayuda a que nuestro modelo pueda encontrar más ocurrencias en los sets de entrenamiento y de prueba, lo que mejorará las predicciones.
* Agregar más características para analizar: por ejemplo incluir una variable que sea la longitud del título o la longitud promedio por palabra.


**Conclusiones finales:** si bien el modelo no resultó muy preciso y requerirá ajustes para poder ser utilizado realmente, pudimos ver todo el proceso de importación de datos, limpieza, muestreo, análisis, vectorización del texto y predicción en una base de datos real. Finalmente lo que vamos a hacer es exportar los datos de la bolsa de palabras con el fin de analizarla en un futuro texto, utilizando un modelo que pueda captar la no linealidad en la relación entre las palabras utilizadas en el título y el número de votos.

In [ ]:
counts.to_csv("/counts.csv")